In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

!pip install transformers
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
import json

import pandas as pd  

     |████████████████████████████████| 4.0 MB 32.5 MB/s 
     |████████████████████████████████| 880 kB 56.2 MB/s 
     |████████████████████████████████| 6.6 MB 53.3 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1c596ff0054c6e0db15fd811e4bdcb71b3fa00970c3d959a78376dc9d98ae3f0
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|BOS|>', eos_token='<|EOS|>', pad_token='<|pad|>') #gpt2-medium


class GPT2Dataset(Dataset):
  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    for txt in txt_list:
      encodings_dict = tokenizer('<|BOS|>'+ txt + '<|EOS|>', truncation=True, max_length=max_length, padding="max_length")
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# import pandas as pd
# xx = pd.read_csv('summary.csv', encoding = "ISO-8859-1")
# print(xx)

file = 'books_dataset.json'
with open(file) as train_file:
    dict_train = json.load(train_file)

# converting json dataset from dictionary to dataframe
train = pd.DataFrame.from_dict(dict_train, orient='index')
train.reset_index(level=0, inplace=True)

xx=train
# data = json.loads(books_dataset)
# df = pd.json_normalize(data['results'])
# xx = df
#print(xx.iloc[:,1])


In [ ]:


summaries = xx.iloc[:,1].copy()

print(summaries)

0      <|BOS|>The Time Machine<|SEP|>time faints his ...
1      <|BOS|>The War of the Worlds<|SEP|>envious eye...
2      <|BOS|>A Princess of Mars<|SEP|>lower atmosphe...
3      <|BOS|>Youth<|SEP|>space aliens tiny dead iden...
4      <|BOS|>2 B R 0 2 B<|SEP|>population control ac...
                             ...                        
280    <|BOS|>Rip Foster Rides the Gray Planet<|SEP|>...
281    <|BOS|>Eastern Standard Tribe<|SEP|>members ps...
282    <|BOS|>Man of Many Minds<|SEP|>minds a secret ...
283    <|BOS|>The Players<|SEP|>aliens not nonsense t...
284    <|BOS|>Rip Foster in Ride the Gray Planet<|SEP...
Name: 0, Length: 285, dtype: object


In [ ]:
dataset = GPT2Dataset(summaries, tokenizer, max_length=768)
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [ ]:
def preprocess_val_dataset(dataset):
  
  s = tokenizer.decode(dataset[0][0]).split('<|SEP|>')
  new_output = s[len(s)-1]
  new_output = s[len(s)-1]
  new_input = s[0]
  for i in range(1,len(s)-1):
    new_input = new_input+'<|SEP|>'+s[i]

  new_input = new_input + '<|SEP|>'
  #print(new_input)
  input_ids = tokenizer.encode(new_input, return_tensors='pt')
  output_ids = tokenizer.encode(new_output, return_tensors='pt')

  return input_ids[0], torch.ones(len(input_ids[0])), output_ids, new_output, new_input

In [ ]:
batch_size=2
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Embedding(50260, 768)

In [ ]:
epochs = 5
learning_rate = 10e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
import time
import datetime
import random
import tensorflow as tf

!pip install ipdb
device = torch.device("cuda")
model.cuda()
total_t0 = time.time()
model = model.to(device)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))
training_stats = []

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        # cc = batch.copy()
        # labels = []
        # for en, ex in enumerate(cc[0]):
        #   import ipdb;ipdb.set_trace()
        #   ind_29 = (ex == 29).nonzero(as_tuple=True)
        #   ind_27 = (ex == 27).nonzero(as_tuple=True)
        #   batch[0][en] = cc[0][en][ind_29[0][1]+1:ind_27[0][2]]
        #   batch[1][en] = cc[1][en][ind_29[0][1]+1:ind_27[0][2]]
        #   labels.append(cc[0][en][ind_29[0][2]+1:])
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    '''
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        input_ids, masks, output_ids, output = preprocess_val_dataset(batch)
        #b_input_ids = batch[0].to(device)
        b_input_ids = input_ids.to(device)
        #b_labels = batch[0].to(device)
        b_labels = input_ids.to(device)
        #b_masks = batch[1].to(device)
        b_masks = masks.to(device)
        #print(f"DEBA: {b_labels}")
        #print(tokenizer.decode(b_input_ids))
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        
    '''
    total_eval_loss = total_train_loss
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

     |████████████████████████████████| 793 kB 22.1 MB/s 
     |████████████████████████████████| 381 kB 65.8 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=bd4dec5fc9d67841b1eac657fc4de9b5fe3c98391a6be991f9255fd856767cf5
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit


======== Epoch 1 / 5 ========
Training...

  Average training loss: 8.15
  Training epoch took: 0:00:48
  Validation Loss: 18.95
  Validation took: 0:00:48

======== Epoch 2 / 5 ========
Training...

  Average training loss: 2.18
  Training epoch took: 0:00:51
  Validation Loss: 5.08
  Validation took: 0:00:51

======== Epoch 3 / 5 ========
Training...

  Average training loss: 1.64
  Training epoch took: 0:00:52
  Validation Loss: 3.81
  Validation took: 0:00:52

======== Epoch 4 / 5 ========
Training...

  Average training loss: 1.24
  Training epoch took: 0:00:52
  Validation Loss: 2.88
  Validation took: 0:00:52

======== Epoch 5 / 5 ========
Training...

  Average training loss: 0.99
  Training epoch took: 0:00:52
  Validation Loss: 2.31
  Validation took: 0:00:52

Training complete!
Total training took 0:04:14 (h:mm:ss)


In [ ]:
model.eval()

prompt = "<|BOS|>we live in solar system<|SEP|> It has nine planets <|SEP|>"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,
                                no_repeat_ngram_size=2,  
                                top_k=5, 
                                max_length = 1000,
                                top_p=0.9, 
                                num_return_sequences=3
                                )
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257,   732,  2107,   287,  6591,  1080,    27,    91,  5188,    47,
            91,    29,   632,   468,  5193, 14705,  1279,    91,  5188,    47,
            91,    29]], device='cuda:0')
0: we live in solar system<|SEP|> It has nine planets <|SEP|>The novel begins at the turn of the 20th century. The first book, written by William Tuning and Michael Ardan, tells the remarkable story of a space ship which was built almost immediately after the first lunar landing. In the aftermath of their maiden voyage, the ship's maiden passengers—Mars, a dying planet, and their children—were killed by an accident and stranded for several hundred light years in a desert.
The story ends when the spaceship is recaptured by the hostile natives of Tars Tarkas, who have taken over their world. There they welcomed the newly renamed Nereids, descendants of an old civilization that once dominated the Solar System. They welcomed by a number of Nokoth natives, including a young prince named Nog. A 

In [ ]:
!pip install rouge/requirements.txt
!pip install rouge-score
from rouge_score import rouge_scorer
rouge_1 = {}
rouge_1["precision"] = []
rouge_1["recall"] = []
rouge_1["fmeasure"] = []

rouge_L = {}
rouge_L["precision"] = []
rouge_L["recall"] = []
rouge_L["fmeasure"] = []


for batch in validation_dataloader:
    input_ids, masks, output_ids, ref_story, prompt = preprocess_val_dataset(batch)
    #b_input_ids = batch[0].to(device)
    b_input_ids = input_ids.to(device)
    #b_labels = batch[0].to(device)
    b_labels = input_ids.to(device)
    #b_masks = batch[1].to(device)
    b_masks = masks.to(device)
    #print(f"DEBA: {b_labels}")
    #print(tokenizer.decode(b_input_ids))
  
    #prompt = input_ids

    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)

    print(generated)

    sample_outputs = model.generate(
                                    generated, 
                                    #bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    no_repeat_ngram_size=2,  
                                    top_k=5, 
                                    max_length = 1000,
                                    top_p=0.9, 
                                    num_return_sequences=3
                                    )
    
    for i, sample_output in enumerate(sample_outputs):
      s =  tokenizer.decode(sample_output, skip_special_tokens=True)
      print(s)
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      scores = scorer.score(s, ref_story)
      print(scores)
      rouge_1["precision"].append(scores["rouge1"][0])
      rouge_1["recall"].append(scores["rouge1"][1])
      rouge_1["fmeasure"].append(scores["rouge1"][2])

      rouge_L["precision"].append(scores["rougeL"][0])
      rouge_L["recall"].append(scores["rougeL"][1])
      rouge_L["fmeasure"].append(scores["rougeL"][2])
      print("************************************")

print("*********ROUGE-1************")
avg_pre = sum(rouge_1["precision"])/len(rouge_1["precision"])
avg_recall = sum(rouge_1["recall"])/len(rouge_1["recall"])
avg_fmeasure = avg_pre = sum(rouge_1["fmeasure"])/len(rouge_1["fmeasure"])
print("avg_acc: "+str(avg_pre))
print("avg_recall: "+str(avg_recall))
print("avg_fmeasure: "+str(avg_fmeasure))

print("*********ROUGE-L************")
avg_pre = sum(rouge_L["precision"])/len(rouge_L["precision"])
avg_recall = sum(rouge_L["recall"])/len(rouge_L["recall"])
avg_fmeasure = avg_pre = sum(rouge_L["fmeasure"])/len(rouge_L["fmeasure"])
print("avg_acc: "+str(avg_pre))
print("avg_recall: "+str(avg_recall))
print("avg_fmeasure: "+str(avg_fmeasure))
      

ERROR: Invalid requirement: 'rouge/requirements.txt'
Hint: It looks like a path. File 'rouge/requirements.txt' does not exist.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257,   220, 50257,   383, 11397, 10318,   690,    27,    91,  5188,
            47,    91,    29,  7785,   699,   584,  9180,  1220,  8982,   262,
          8806,  6253, 11661, 13069,  3653, 17372,  5328, 31707, 27406,    12,
            88,   418,   407, 34579,  4678, 12524,  4894, 16343,   284,  1842,
           257,  9233,    27,    91,  5188,    47,    91,    29]],
       device='cuda:0')
  The Planet Savers<|SEP|>various other Jay / Jason the personality doctor volunteers Trailmen fever hypnosis yo-yos not External skills hostile heat nest to love a secondary<|SEP|>The planet Saverns has been retconned by Jason Allison, a ruthless advertising executive who knows his place in the world and can spot a flaw in its advertising tactics. Jason has hired an undercover FBI agent to infiltrate the planet's political and economic life, and has installed a sophisticated advertising program to fool its unsuspecting consumers. In this guise, Jason is a regular guy who volunteers his

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge1': Score(precision=0.23887973640856672, recall=0.19594594594594594, fmeasure=0.21529324424647364), 'rougeL': Score(precision=0.09884678747940692, recall=0.08108108108108109, fmeasure=0.08908685968819599)}
************************************
tensor([[50257,   220, 50257,   383,  3878, 18329,  6896,    27,    91,  5188,
            47,    91,    29, 28116,   282,  2585,  1812,  1956,  1049,  7630,
          5417,  1241,  1660,   262, 26824,  1017,  1191,   257,   649, 37874,
          3450,   661,   636,  4025,  8046,    27,    91,  5188,    47,    91,
            29]], device='cuda:0')
  The Great Nebraska Sea<|SEP|>several states State land great profit sea level water the governors slivers a new inland nature people part larger fault<|SEP|>The story begins with the discovery of a submerged land far larger than the Mississippi, called the Great Nebes. The story mainly takes place in the present-day West Indies, whereof the great bulk of the world was once located. It is a land

...

{'rouge1': Score(precision=0.5334476843910806, recall=0.4146666666666667, fmeasure=0.4666166541635409), 'rougeL': Score(precision=0.1646655231560892, recall=0.128, fmeasure=0.14403600900225058)}
************************************
tensor([[50257,   220, 50257,   383,   370, 11608, 38484,  1868,    27,    91,
          5188,    47,    91,    29, 37004, 10425,   664,    78,   359,   408,
         16790,  1402,  2042, 34896, 23441,  7183,  2769,  2272,  1528, 48851,
          6439,  3777,  1029,  1524,   257,  6737,  1657,    27,    91,  5188,
            47,    91,    29]], device='cuda:0')
  The Wailing Asteroid<|SEP|>radio signals recoilless pistol small black cubes cube readers deep space days interstellar unknown weapons high school a signal light<|SEP|>The novel concerns the adventures of the Wounded Soldier and the constabulary, during the Cold War between the Soviet Union and North America.


The story is told in flashbacks, as if the reader were reading the novel at the time.

I

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge1': Score(precision=0.4654895666131621, recall=0.36295369211514394, fmeasure=0.4078762306610408), 'rougeL': Score(precision=0.17014446227929375, recall=0.13266583229036297, fmeasure=0.14908579465541494)}
************************************
tensor([[50257,   220, 50257,  1160,    11,   830,  1004,  6120,   739,   262,
          6896,    27,    91,  5188,    47,    91,    29,  6767,  3493,  7937,
         14101,  3048,  3819, 11000,  1204, 13435, 17219,   284,  9161, 36945,
           418, 14052,  1103,   220,  2846, 11561,  3200,   257, 11428,  5417,
          9234,    27,    91,  5188,    47,    91,    29]], device='cuda:0')
  20,000 Leagues under the Sea<|SEP|>Teacher ships devastating effects Other Marine life Advanced biology to investigate nemos motivation real  terms commander secret a mysterious sea monster<|SEP|>During a visit to the fictional submerged land of Atlantis, Dr. Pierre Aronnax, a French marine biologist and narrator of the story, receives a last-minute invit

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge1': Score(precision=0.5, recall=0.37617135207496655, fmeasure=0.4293353705118411), 'rougeL': Score(precision=0.17259786476868327, recall=0.12985274431057564, fmeasure=0.14820473644003057)}
************************************
tensor([[50257,   220, 50257, 10467, 11397,    27,    91,  5188,    47,    91,
            29,    70,   328,  5109,  6129,   373,   862, 10620, 25972,  3352,
         19435,   262,  2546,   286,   257, 17481, 27842,  4695, 25321,   262,
          1675,  3953, 33712,  9441,  5384,  1204,   257,   340,  6265, 40663,
            27,    91,  5188,    47,    91,    29]], device='cuda:0')
  Third Planet<|SEP|>gigantic fly wasps sizes chickens plans insects the size of a wolf ants animals descendants the To measure microbes survival humans life a it broke spaceship<|SEP|>The novel concerns a group of humans who build a spaceship and then try to scale it using insects.  Although the aliens succeed, the survivalist volunteers to host two large, independent animals o